<h1>Peer-Graded Assignment on Toronto Neighbourhoods<h1>

# Importing/Installing Necessary Packages

In [12]:
import pandas as pd
import numpy as np
!pip install beautifulsoup4
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

import requests
!pip install wget
import wget
import urllib.request
from urllib.request import urlopen

!pip install geopy
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as plt

<p> Reading in Wikipedia Page and Converting to Pandas Dataframe <p>

In [13]:
url = "http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [14]:
df=pd.read_html(url,header=0,flavor='bs4')
df1=df[0]
ca_df=pd.DataFrame(df1,columns=['Postal Code','Borough','Neighbourhood'])
ca_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<p> Removing "Not Assigned" values from table <p>

In [15]:
ca_df.drop(ca_df[ca_df.Borough=='Not assigned'].index,inplace=True)

In [16]:
ca_df.shape

(103, 3)

Read in CSV file with Longitude and Latitude

In [17]:
ca_coord=pd.read_csv('http://cocl.us/Geospatial_data')
ca_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_tca=pd.merge(ca_df,ca_coord,on='Postal Code',how='left')
print(df_tca)

    Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4          Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98

Prints Shape of Dataframe df_tca

In [19]:
df_tca.shape

(103, 5)

Create Dataframe of Boroughs amalgamated into City of Toronto 2016

In [20]:
tca = df_tca[df_tca['Borough'].str.contains('Toronto')].reset_index(drop=True)
tca.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [22]:
tca.shape

(39, 5)

Create Map of Toronto with Labels for each Postal Code <br/>
Format: Neighbourhood (Borough)

In [23]:
add_tca='Toronto,CA'
geolocator=Nominatim(user_agent="Foursquare_agent")
location=geolocator.geocode(add_tca)
t_lat=location.latitude
t_long=location.longitude

In [24]:
map_tca=folium.Map(location=[location.latitude,location.longitude],zoom_start=11)

for lat,long,neighbour,borough in zip(df_tca.Latitude,df_tca.Longitude,df_tca.Neighbourhood,df_tca.Borough):
    label='{} ({})'.format(neighbour,borough)
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat,long],
        radius=3,
        color='blue',
        popup=label,
        fill=False
    ).add_to(map_tca)
    
map_tca

Map of Outskirt Boroughs of Toronto

In [30]:
new_toronto=folium.Map(location=[location.latitude,location.longitude],zoom_start=11)

for lat,long,neighbour,borough in zip(tca.Latitude,tca.Longitude,tca.Neighbourhood,tca.Borough):
    label='{} ({})'.format(neighbour,borough)
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat,long],
        radius=3,
        color='green',
        popup=label,
        fill=True, fill_color='green',fill_opacity=1
    ).add_to(new_toronto)
    
new_toronto

## Four Square API: Neighbourhood Venue Analysis ##

Credentials

In [31]:
CLIENT_ID = 'J3BXVIECWLAHZXARVRCBB2YM5UHI2AMJURPE1GOOMAJEF4RK' # your Foursquare ID
CLIENT_SECRET = 'JNVENONWCBDNZZ3UG5RTBGJ5TXMCXOPWC0GL3VIQ0AAAG0MC' # your Foursquare Secret
VERSION = '20200806' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J3BXVIECWLAHZXARVRCBB2YM5UHI2AMJURPE1GOOMAJEF4RK
CLIENT_SECRET:JNVENONWCBDNZZ3UG5RTBGJ5TXMCXOPWC0GL3VIQ0AAAG0MC


-

New Function "getNearbyVenues" returns venues within the radius of each neighbourhood's Coorindates

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
LIMIT=100
tca_venues=getNearbyVenues(names=tca.Neighbourhood,
                                   latitudes=tca.Latitude,
                                   longitudes=tca.Longitude)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [35]:
tca_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [36]:
tca_venues.shape

(1624, 7)

In [37]:
tca_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


Using One-Hot Encoding to format Venue Category for easier analysis

In [38]:
# one hot encoding
tca_onehot = pd.get_dummies(tca_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tca_onehot['Neighbourhood'] = tca_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [tca_onehot.columns[-1]] + list(tca_onehot.columns[:-1])
tca_onehot = tca_onehot[fixed_columns]

Calculating frequency of each category for each neighbourhood

In [39]:
tca_grouped=tca_onehot.groupby('Neighbourhood').mean().reset_index()

In [40]:
top_count=5

for hood in tca_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tca_grouped[tca_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['Venue','Freq']
    temp = temp.iloc[1:]
    temp['Freq'] = temp['Freq'].astype(float)
    temp = temp.round({'Freq': 2})
    print(temp.sort_values('Freq', ascending=False).reset_index(drop=True).head(top_count))
    print('\n')

----Berczy Park----
            Venue  Freq
0     Coffee Shop  0.10
1        Beer Bar  0.03
2  Farmers Market  0.03
3            Café  0.03
4     Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
                Venue  Freq
0                Café  0.13
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.04
4   Convenience Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  Venue  Freq
0    Light Rail Station  0.13
1           Pizza Place  0.07
2            Comic Shop  0.07
3                  Park  0.07
4  Gym / Fitness Center  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                Venue  Freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2    Airport Terminal  0.12
3       Boat or Ferry  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                Venue  F

New function to return most common venue type for each neighbourhood.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Calculating most common venue type for each neighborhood in Toronto

In [87]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tca_grouped['Neighbourhood']

for ind in np.arange(tca_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tca_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Café,Cocktail Bar,Cheese Shop,Museum
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Gym,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden Center,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Auto Workshop,Garden,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Department Store,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Korean Restaurant


## Begin K-Means Clustering ##

In [88]:
tca_cluster=tca_grouped.drop(['Neighbourhood'],1)

Identify # of Clusters (8), perform clustering, and attach cluster labels to new df 

In [89]:
k_clusters=8
kmeans=KMeans(n_clusters=k_clusters,n_init=20,max_iter=100,random_state=1).fit(tca_cluster)

In [90]:
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

In [91]:
df_merge=tca
df_merge=df_merge.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
df_merge['Cluster Labels']=df_merge['Cluster Labels'].astype(int)
df_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,5,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,5,Coffee Shop,Diner,Yoga Studio,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,5,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Diner,Ramen Restaurant,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,5,Coffee Shop,Café,Clothing Store,Cosmetics Shop,Restaurant,Cocktail Bar,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio


In [92]:
df_merge.head(35)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,5,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,5,Coffee Shop,Diner,Yoga Studio,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,5,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Diner,Ramen Restaurant,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,5,Coffee Shop,Café,Clothing Store,Cosmetics Shop,Restaurant,Cocktail Bar,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,5,Coffee Shop,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Café,Cocktail Bar,Cheese Shop,Museum
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,5,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Department Store,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Korean Restaurant
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Restaurant,Diner,Italian Restaurant,Candy Store,Baby Store,Nightclub
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,5,Coffee Shop,Café,Gym,Restaurant,Clothing Store,Hotel,Bar,Thai Restaurant,Steakhouse,Sushi Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Bakery,Pharmacy,Middle Eastern Restaurant,Pet Store,Smoke Shop,Café,Bar,Bank,Supermarket,Brewery


In [93]:
# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Neighbourhood'], df_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [94]:
df_merge.loc[df_merge['Cluster Labels'] == 0, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Restaurant,Diner,Italian Restaurant,Candy Store,Baby Store,Nightclub
9,West Toronto,0,Bakery,Pharmacy,Middle Eastern Restaurant,Pet Store,Smoke Shop,Café,Bar,Bank,Supermarket,Brewery
22,West Toronto,0,Mexican Restaurant,Café,Thai Restaurant,Discount Store,Diner,Speakeasy,Bar,Flea Market,Bakery,Italian Restaurant
24,Central Toronto,0,Sandwich Place,Café,Coffee Shop,BBQ Joint,History Museum,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Flower Shop,Pub
27,Downtown Toronto,0,Café,Restaurant,Bar,Japanese Restaurant,Bookstore,Sandwich Place,Bakery,Beer Bar,Beer Store,Italian Restaurant
30,Downtown Toronto,0,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Mexican Restaurant,Bar,Burger Joint,Pizza Place,Dessert Shop,Park


In [95]:
df_merge.loc[df_merge['Cluster Labels'] == 1, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Garden,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [96]:
df_merge.loc[df_merge['Cluster Labels'] == 2, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,East Toronto,2,Park,Pizza Place,Sandwich Place,Sushi Restaurant,Ice Cream Shop,Fish & Chips Shop,Fast Food Restaurant,Italian Restaurant,Liquor Store,Movie Theater
20,Central Toronto,2,Gym,Hotel,Park,Sandwich Place,Food & Drink Shop,Department Store,Breakfast Spot,Pizza Place,Gym / Fitness Center,Concert Hall
38,East Toronto,2,Light Rail Station,Garden Center,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Auto Workshop,Garden,Skate Park


In [97]:
df_merge.loc[df_merge['Cluster Labels'] == 3, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Bus Line,Swim School,Dim Sum Restaurant,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
21,Central Toronto,3,Park,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [73]:
df_merge.loc[df_merge['Cluster Labels'] == 4, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,4,Park,Restaurant,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
33,Downtown Toronto,4,Park,Playground,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [98]:
df_merge.loc[df_merge['Cluster Labels'] == 5, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,5,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,Downtown Toronto,5,Coffee Shop,Diner,Yoga Studio,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Hobby Shop,Fried Chicken Joint
2,Downtown Toronto,5,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Diner,Ramen Restaurant,Pizza Place
3,Downtown Toronto,5,Coffee Shop,Café,Clothing Store,Cosmetics Shop,Restaurant,Cocktail Bar,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant
5,Downtown Toronto,5,Coffee Shop,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Café,Cocktail Bar,Cheese Shop,Museum
6,Downtown Toronto,5,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Department Store,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Korean Restaurant
8,Downtown Toronto,5,Coffee Shop,Café,Gym,Restaurant,Clothing Store,Hotel,Bar,Thai Restaurant,Steakhouse,Sushi Restaurant
10,Downtown Toronto,5,Coffee Shop,Aquarium,Café,Hotel,Brewery,Scenic Lookout,Italian Restaurant,Restaurant,Fried Chicken Joint,Plaza
11,West Toronto,5,Bar,Coffee Shop,Asian Restaurant,Café,Restaurant,Men's Store,Vietnamese Restaurant,Italian Restaurant,Brewery,Japanese Restaurant
12,East Toronto,5,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Indian Restaurant,Spa,Caribbean Restaurant


In [99]:
df_merge.loc[df_merge['Cluster Labels'] == 6, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,6,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio


In [100]:
df_merge.loc[df_merge['Cluster Labels'] == 7, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,7,Park,Restaurant,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [101]:
df_merge.loc[df_merge['Cluster Labels'] == 8, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# Analysis Summary #

Initial runs using a KMeans=5 returned all but a few neighbourhoods falling into one category. As K-Means was increased (to 8) the distribution of neighbourhoods was much more even. 

Cluster # 5 leans heavily towards food and beverage services such as restaurants, bars, and markets. Interesting have strong pull here when considering areas with higher frequencies of bars and restaurants generally have fewer markets as people are eating out more than dining at home. 

I am surprised to see a number of categories with only one neighborhood that all share common venues, specifically Clusters 3,4,6, and 7. All show a strong importance on being outdoors or exercising with Parks or trails being the first or second most common venues in these neighborhoods.

